# Week 2 Day 2
We're going to build a simple Agent system for generating cold outreach emails:
    1. Agent workflow'
    2. Use of tools to call functions
    3. Agent collaboration via Tools and Handoffs

In [26]:
# Necessary Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from pydantic import BaseModel

In [3]:
# Load environment
load_dotenv(override=True)

True

# Step 1: Agent WorkFlow

In [4]:
# Create Agent Instructions
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# Create Agents for each instruction

model_name="gpt-4o-mini"

# Professional Sales Agent
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model_name
)

# Engaging Sales Agent
sales_agent2 = Agent(
    "Engaging Sales Agent",
    instructions=instructions2,
    model=model_name
)

# Busy Sales Agent
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model_name
)


In [6]:
# Will use streaming API (.run_streamed())
# Run the instructions but call Runner.run_streamed(agent, instructions) in order to stream back results
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

# Runner.run_streamed(agent, instructions) returns a coroutine object so we have to iterate through it 
# and call async in the for loop
async for event in result.stream_events():
    # Need to check that actual text was returned that we can print
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a cutting-edge SaaS solution designed to simplify SOC 2 compliance and streamline the audit process.

In today's fast-paced environment, achieving and maintaining compliance can be a complex and time-consuming endeavor. Our AI-powered platform automates critical compliance tasks, helps identify potential gaps, and prepares your organization for audits with efficiency and confidence.

With ComplAI, you can expect:
- **Automated Compliance Checks**: Reduce manual effort and minimize human error.
- **Real-Time Reporting**: Gain insights into compliance status at your fingertips.
- **Scalability**: Easily adapt our solution as your organization expands.

I would love to schedule a brief call to discuss how ComplAI can support your efforts in achieving SOC 2 compliance more efficiently. Are you available for a qui

In [7]:
# Going to use Asyncio for parallel agent calliing

message = "Send a cold sales email."

# Use trace to run agents asynchronously
with trace("Parallel Cold Emails"):
    # Will return a collection of results
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Store outputs
outputs = [result.final_output for result in results]

# Iterate through outputs and print
for output in outputs:
    print(output + "\n\n")

Subject: Simplify SOC 2 Compliance and Audit Preparation with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. 

As organizations increasingly prioritize data security and industry standards, achieving and maintaining SOC 2 compliance can be a daunting task. At ComplAI, we offer an AI-powered SaaS solution specifically designed to streamline SOC 2 compliance and prepare your team for audits with ease.

Our platform helps you:

- Automate compliance processes, saving valuable time and resources.
- Access intuitive audits and reporting tools tailored to your business needs.
- Ensure continuous monitoring and real-time updates on compliance status.

With ComplAI, you can focus on what you do best—growing your business—while we handle the complexities of compliance management.

I would love the opportunity to discuss how we can support your compliance efforts. Are you available for a brief call next week?

Thank you for considering ComplAI. I look forward to the possi

In [8]:
# Create an agent that picks the best cold email
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only.",
    model=model_name
)

In [9]:
# Define the message for Runner.run()
message = "Write a cold sales email"

# Run the agents asynchronously
with trace("Selection from sales people"):
    # Returns a list of coroutines
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Gather the results into one list 
outputs = [result.final_output for result in results]

# Join the email strings from the 'outputs' list
emails = "Cold Sales Emails:\n\n".join(outputs)

# Run the sales picker to pick the best email
best_email = await Runner.run(sales_picker, emails)

# print out the best email
print(f"Best sales email:\n{best_email.final_output}")

Best sales email:
Subject: Stop Chasing Compliance – Let Us Do the Heavy Lifting!

Hi [Recipient's Name],

If you've ever thought that getting SOC2 compliance feels a little like trying to herd cats—let me be your cat herder (and I promise, I’m better at this than actual cat herding).

Here at ComplAI, we've crafted a SaaS tool that practically spoons you into compliance nirvana while preparing for audits. Imagine sitting back with a cup of coffee, watching compliance tasks get done as if by magic (the good kind—no rabbits or top hats involved).

Our AI-driven platform:

- **Automates** the chasing of documents (because no one wants to play hide and seek with paperwork).
- **Simplifies** your audit prep (no more late-night panic snacks and existential crises).
- **Ensures** that you're not just compliant, but in compliance *style* (because who said compliance can't be chic?).

Let’s have a chat! I promise it won’t be as boring as your last team meeting. Hit reply, and I’ll make sure to

# Step 2 and 3: Tools and Agent interactions
- Will now add tools to the mix

- Wrap function with the @function_tool decorator

In [10]:
# Define a function as a tool to send an email using SendGrid

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    # Set the from email
    from_email = Email("twostrokes210business@gmail.com")
    # Set the to email
    to_email = To("twostrokes210business@gmail.com")
    # Set the email body content
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return{"status": "success"}


### This has automatically been converted into a tool, with the JSON boilerplate created

In [11]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F535863560>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [12]:
# Convert agent into a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F535F19080>, strict_json_schema=True, is_enabled=True)

### So now we can gather all he tools together:
A tool fo reach of our 3 email-writing agents

And a tool for our function to send emails

In [13]:
# Create description
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F535D7BC40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F533BC4E00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

## Now it's time for our Sales Manager - our planning agent!

In [14]:
# Create instructions for agent
instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. You never generate sales emails yourself; you always use the tools. You try all 3 sales_agent tools once before choosing the best one. You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

# Create the sales agent with tools
sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)
# Create a messge for the agent to execute
message = "Send a cold sales email address to 'Dear CEO'"

# Define a trace for the agent
with trace("Sales_manager trace"):
    result = await Runner.run(sales_manager, message)
    print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    I've successfully sent the cold sales email addressed to 'Dear CEO'. If you need anything else, feel free to ask!
- 13 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!

### Handoffs represent a way an agent can delegate to an agent, passing control to it
Handoffs and Agents-as-tools are similar:
- In both cases, an Agent can collaborate with another Agent
- With tools, control passes back
- With handoffs, control passes acrosss

In [15]:
# Subject instructions
subject_instructions = "You can write a subject for a cold sales email. You are given a message and you need to write a subject for an email that is likely to get a response."

# HTML instructions
html_instructions = "You can convert a text email body to an HTML email body. You are given a text email body which might have some markdown and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Create subject_writer agent
subject_writer = Agent(
    name="Email subject writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)
# Use the subject_writer agent to create a tool
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

# Create a html_converter agent
html_converter = Agent(
    name="HTML email body converter",
    instructions=html_instructions,
    model="gpt-4o-mini"
)
# Use the html_converter agent to create a tool
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [ ]:
# Define function tool to send email
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with th egiven subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email  = Email("twostrokes210business@gmail.com")
    to_email = To("twostrokes210business@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)

    return{"status": "success"}


In [17]:
# Define tools list
tools = [subject_tool, html_tool, send_html_email]

In [18]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F535F1A160>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F533BE47C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with th egiven subject and HTML body to 

In [19]:
# Create email agent instructions
instructions = "You are an email formatter and sender. You receive the body of an email to be sent. You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. Finally, you use the send_html_email tool to send the email with the subject and HTML body."

# Create email agent with handoff description
emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

### Now we have 3 tools and 1 handoff

In [24]:
# Define the list of tools
tools = [tool1, tool2, tool3]

#handoffs  = [emailer_agent]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F535D7BC40>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F533BC4E00>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

# Define a sales_manager agent with tools and handoffs
sales_manager = Agent(
    name="Sales manager",
    instructions=sales_manager_instructions,
    handoffs=handoffs,
    tools=tools,
    model="gpt-4o-mini"
)
# Defind a message for the sales_manager agent
message = "Send out a cold sales email addressed to Dear CEO from Alice"

# Add a "Automated SDR" trace
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [ ]:
sales_manager = Agent(
    instructions=sales_manager_instructions,
    handoffs=handoffs,
    tools=tools,
    model="gpt-4o-mini"
)
# Defind a message for the sales_manager agent
message = "Send out a cold sales email addressed to Dear CEO from Alice"

# Add a "Automated SDR" trace
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [ ]:
# Define a structured output class
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# Create guardrail agent that checks if the user is including someone's personal name in what they want the agent to do
# Need to define instructions that include some type of guarding, as well as the output type

guardrail_agent = Agent(
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    # This is telling the agent that we don't want it to output text, rather, output an object that conforms to our defined schema
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [28]:
# Now we need to define a guardrail coroutine
# Coroutines are always need to have async before the def
# There are 3 types of guardrail types: input guardrails, output guardrails, and tripwires
# Guardrails always need to return GuardrailFunctionOutput
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)


In [29]:
# Now create an agent that implements the guardrail, tools, and handoffs
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
)

# Message to be sent that the guardral will check
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire